<a href="https://colab.research.google.com/github/WinNatch/Modelling-Individual-Assessment/blob/main/Q1_Individual_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Intall and Import Package/Library
!pip install pulp
from pulp import *

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 26.4 MB/s eta 0:00:00


##Question A

In [ ]:
# Define Environment & Direction of Optimization
model = LpProblem("Q1", sense=LpMinimize)

# List of Product
product = ['A', 'B', 'C']

# List (TimePeriods)
month = [0,1,2,3,4]

# Parameters and Data
A_demand = {1:100, 2:110, 3:120, 4:130} # Demand data per time period
B_demand = {1:80, 2:90, 3:100, 4:110} # Demand data per time period
C_demand = {1:120, 2:130, 3:140, 4:150} # Demand data per time period
UPC = {'A':20, 'B':30, 'C':10} # Unit Production Cost per time period
UHC = {'A':1, 'B':2, 'C':0.5} # Unit Holding Cost per time period
UPH = {'A':1.2, 'B':1.3, 'C':1} # Production hour per unit
TPC = {1:500, 2:450, 3:400, 4:350} # Production Capacity per time period

# Desicion Variables
Pt = LpVariable.dicts("Quantity", [(i,j) for i in product for j in month], lowBound=0, cat='Integer')
It = LpVariable.dicts("Inventory", [(i,j) for i in product for j in month], lowBound=0, cat='Integer')

# Objective Function
model += (lpSum(UPC[i]*Pt[i,j] for i in product for j in month) + lpSum(UHC[i]*It[i,j] for i in product for j in month))

# Constraints
for i in product:
    model += It[i,0] == 0

for j in month[1:]:
    model += (Pt['A', j] + It['A', j-1] - It['A', j]) == A_demand[j] # A_Inventory-Balancing Constraints
    model += (Pt['B', j] + It['B', j-1] - It['B', j]) == B_demand[j] # B_Inventory-Balancing Constraints
    model += (Pt['C', j] + It['C', j-1] - It['C', j]) == C_demand[j] # C_Inventory-Balancing Constraints
    model += lpSum(UPH[i] * Pt[i,j] for i in product) <= TPC[j] # Production Hour Capacity Constraints

for i in product:
    model += It[i,4] == 0 # To Ensure no Remaing Invetory Left

# Solve the Problem
model.writeLP("Q1.lp")
model.solve()

# The Status of the Solution
print("Model Status:{}".format(LpStatus[model.status]))

# To Display Optimal Decision Variables & Reduced Cost per Variable
for v in model.variables():
	  print(v.name, "=", v.varValue)

# To Display Optimal Value of Objective Function
print("Objective=", value(model.objective))

Model Status:Optimal
Inventory_('A',_0) = 0.0
Inventory_('A',_1) = 0.0
Inventory_('A',_2) = 0.0
Inventory_('A',_3) = 0.0
Inventory_('A',_4) = 0.0
Inventory_('B',_0) = 0.0
Inventory_('B',_1) = 0.0
Inventory_('B',_2) = 0.0
Inventory_('B',_3) = 0.0
Inventory_('B',_4) = 0.0
Inventory_('C',_0) = 0.0
Inventory_('C',_1) = 42.0
Inventory_('C',_2) = 113.0
Inventory_('C',_3) = 99.0
Inventory_('C',_4) = 0.0
Quantity_('A',_0) = 0.0
Quantity_('A',_1) = 100.0
Quantity_('A',_2) = 110.0
Quantity_('A',_3) = 120.0
Quantity_('A',_4) = 130.0
Quantity_('B',_0) = 0.0
Quantity_('B',_1) = 80.0
Quantity_('B',_2) = 90.0
Quantity_('B',_3) = 100.0
Quantity_('B',_4) = 110.0
Quantity_('C',_0) = 0.0
Quantity_('C',_1) = 162.0
Quantity_('C',_2) = 201.0
Quantity_('C',_3) = 126.0
Quantity_('C',_4) = 51.0
Objective= 26127.0


In [ ]:
from pulp import LpStatus, value

# The Status of the Solution
print("Model Status:{}".format(LpStatus[model.status]))

# Initialize dictionaries to store inventory and quantity values
inventory_values = {}
quantity_values = {}

# Store Optimal Decision Variables
for v in model.variables():
    if "_0" not in v.name:
        if "Inventory" in v.name:
            inventory_values[v.name] = v.varValue
        elif "Quantity" in v.name:
            quantity_values[v.name] = v.varValue

# Extract keys for quantities and inventories
quantity_keys = sorted(quantity_values.keys())
inventory_keys = sorted(inventory_values.keys())

# Print the results in separate columns for quantities and inventories
print("\nOptimal Assignment Plan")
print(f"{'Quantity':<31} {'Inventory'}")
print(f"{'-'*31} {'-'*30}")

max_len = max(len(quantity_keys), len(inventory_keys))

# Print the table with Quantity and Inventory in separate columns
for i in range(max_len):
    quantity_key = quantity_keys[i] if i < len(quantity_keys) else ""
    inventory_key = inventory_keys[i] if i < len(inventory_keys) else ""

    quantity_value = quantity_values.get(quantity_key, "")
    inventory_value = inventory_values.get(inventory_key, "")

    print(f"{quantity_key:<20} {quantity_value:<10} {inventory_key:<20} {inventory_value}")

# Display Optimal Value of Objective Function
print("\nTotal Cost = £", value(model.objective))


Model Status:Optimal

Optimal Assignment Plan
Quantity                        Inventory
------------------------------- ------------------------------
Quantity_('A',_1)    100.0      Inventory_('A',_1)   0.0
Quantity_('A',_2)    110.0      Inventory_('A',_2)   0.0
Quantity_('A',_3)    120.0      Inventory_('A',_3)   0.0
Quantity_('A',_4)    130.0      Inventory_('A',_4)   0.0
Quantity_('B',_1)    80.0       Inventory_('B',_1)   0.0
Quantity_('B',_2)    90.0       Inventory_('B',_2)   0.0
Quantity_('B',_3)    100.0      Inventory_('B',_3)   0.0
Quantity_('B',_4)    110.0      Inventory_('B',_4)   0.0
Quantity_('C',_1)    162.0      Inventory_('C',_1)   42.0
Quantity_('C',_2)    201.0      Inventory_('C',_2)   113.0
Quantity_('C',_3)    126.0      Inventory_('C',_3)   99.0
Quantity_('C',_4)    51.0       Inventory_('C',_4)   0.0

Total Cost = £ 26127.0


##Question B

In [ ]:
# Define Environment & Direction of Optimization
model = LpProblem("Q1", sense=LpMinimize)

# List of Product
product = ['A', 'B', 'C']

# List (TimePeriods)
month = [0,1,2,3,4]

# Parameters and Data
A_demand = {1:100, 2:110, 3:120, 4:130} # Demand data per time period
B_demand = {1:80, 2:90, 3:100, 4:110} # Demand data per time period
C_demand = {1:120, 2:130, 3:140, 4:150} # Demand data per time period
UPC = {'A':20, 'B':30, 'C':10} # Unit Production Cost per time period
UHC = {'A':1, 'B':2, 'C':0.5} # Unit Holding Cost per time period
UPH = {'A':1.2, 'B':1.3, 'C':1} # Production hour per unit
TPC = {1:500, 2:450, 3:400, 4:350} # Production Capacity per time period

# Desicion Variables
Pt = LpVariable.dicts("Quantity", [(i,j) for i in product for j in month], lowBound=0, cat='Integer')
It = LpVariable.dicts("Inventory", [(i,j) for i in product for j in month], lowBound=0, cat='Integer')

# Objective Function
model += (lpSum(UPC[i]*Pt[i,j] for i in product for j in month) + lpSum(UHC[i]*It[i,j] for i in product for j in month)) - (UPC['B']*Pt['B',3]) + (35*Pt['B',3])

# Constraints
for i in product:
    model += It[i,0] == 0

for j in month[1:]:
    model += (Pt['A', j] + It['A', j-1] - It['A', j]) == A_demand[j] # A_Inventory-Balancing Constraints
    model += (Pt['B', j] + It['B', j-1] - It['B', j]) == B_demand[j] # B_Inventory-Balancing Constraints
    model += (Pt['C', j] + It['C', j-1] - It['C', j]) == C_demand[j] # C_Inventory-Balancing Constraints
    model += lpSum(UPH[i] * Pt[i,j] for i in product) <= TPC[j] # Production Hour Capacity Constraints

for i in product:
    model += It[i,4] == 0

# Solve the Problem
model.writeLP("Q1.lp")
model.solve()

# The Status of the Solution
print("Model Status:{}".format(LpStatus[model.status]))

# To Display Optimal Decision Variables & Reduced Cost per Variable
for v in model.variables():
	  print(v.name, "=", v.varValue)

# To Display Optimal Value of Objective Function
print("Objective=", value(model.objective))

Model Status:Optimal
Inventory_('A',_0) = 0.0
Inventory_('A',_1) = 0.0
Inventory_('A',_2) = 0.0
Inventory_('A',_3) = 0.0
Inventory_('A',_4) = 0.0
Inventory_('B',_0) = 0.0
Inventory_('B',_1) = 0.0
Inventory_('B',_2) = 100.0
Inventory_('B',_3) = 0.0
Inventory_('B',_4) = 0.0
Inventory_('C',_0) = 0.0
Inventory_('C',_1) = 59.0
Inventory_('C',_2) = 0.0
Inventory_('C',_3) = 99.0
Inventory_('C',_4) = 0.0
Quantity_('A',_0) = 0.0
Quantity_('A',_1) = 100.0
Quantity_('A',_2) = 110.0
Quantity_('A',_3) = 120.0
Quantity_('A',_4) = 130.0
Quantity_('B',_0) = 0.0
Quantity_('B',_1) = 80.0
Quantity_('B',_2) = 190.0
Quantity_('B',_3) = 0.0
Quantity_('B',_4) = 110.0
Quantity_('C',_0) = 0.0
Quantity_('C',_1) = 179.0
Quantity_('C',_2) = 71.0
Quantity_('C',_3) = 239.0
Quantity_('C',_4) = 51.0
Objective= 26279.0


In [ ]:
from pulp import LpStatus, value

# The Status of the Solution
print("Model Status:{}".format(LpStatus[model.status]))

# Initialize dictionaries to store inventory and quantity values
inventory_values = {}
quantity_values = {}

# Store Optimal Decision Variables
for v in model.variables():
    if "_0" not in v.name:
        if "Inventory" in v.name:
            inventory_values[v.name] = v.varValue
        elif "Quantity" in v.name:
            quantity_values[v.name] = v.varValue

# Extract keys for quantities and inventories
quantity_keys = sorted(quantity_values.keys())
inventory_keys = sorted(inventory_values.keys())

# Print the results in separate columns for quantities and inventories
print("\nOptimal Assignment Plan")
print(f"{'Quantity':<31} {'Inventory'}")
print(f"{'-'*31} {'-'*30}")

max_len = max(len(quantity_keys), len(inventory_keys))

# Print the table with Quantity and Inventory in separate columns
for i in range(max_len):
    quantity_key = quantity_keys[i] if i < len(quantity_keys) else ""
    inventory_key = inventory_keys[i] if i < len(inventory_keys) else ""

    quantity_value = quantity_values.get(quantity_key, "")
    inventory_value = inventory_values.get(inventory_key, "")

    print(f"{quantity_key:<20} {quantity_value:<10} {inventory_key:<20} {inventory_value}")

# Display Optimal Value of Objective Function
print("\nTotal Cost = £", value(model.objective))

Model Status:Optimal

Optimal Assignment Plan
Quantity                        Inventory
------------------------------- ------------------------------
Quantity_('A',_1)    100.0      Inventory_('A',_1)   0.0
Quantity_('A',_2)    110.0      Inventory_('A',_2)   0.0
Quantity_('A',_3)    120.0      Inventory_('A',_3)   0.0
Quantity_('A',_4)    130.0      Inventory_('A',_4)   0.0
Quantity_('B',_1)    80.0       Inventory_('B',_1)   0.0
Quantity_('B',_2)    190.0      Inventory_('B',_2)   100.0
Quantity_('B',_3)    0.0        Inventory_('B',_3)   0.0
Quantity_('B',_4)    110.0      Inventory_('B',_4)   0.0
Quantity_('C',_1)    179.0      Inventory_('C',_1)   59.0
Quantity_('C',_2)    71.0       Inventory_('C',_2)   0.0
Quantity_('C',_3)    239.0      Inventory_('C',_3)   99.0
Quantity_('C',_4)    51.0       Inventory_('C',_4)   0.0

Total Cost = £ 26279.0


##Question C

In [ ]:
# Define Environment & Direction of Optimization
model = LpProblem("Q1", sense=LpMinimize)

# List of Product
product = ['A', 'B', 'C']

# List (TimePeriods)
month = [0,1,2,3,4]

# Parameters and Data
A_demand = {1:100, 2:110, 3:120, 4:130} # Demand data per time period
B_demand = {1:80, 2:90, 3:100, 4:110} # Demand data per time period
C_demand = {1:120, 2:130, 3:140, 4:170} # Demand data per time period (C in month 4th in crease by 20)
UPC = {'A':20, 'B':30, 'C':10} # Unit Production Cost per time period
UHC = {'A':1, 'B':2, 'C':0.5} # Unit Holding Cost per time period
UPH = {'A':1.2, 'B':1.3, 'C':1} # Production hour per unit
TPC = {1:500, 2:450, 3:400, 4:350} # Production Capacity per time period

# Desicion Variables
Pt = LpVariable.dicts("Quantity", [(i,j) for i in product for j in month], lowBound=0, cat='Integer')
It = LpVariable.dicts("Inventory", [(i,j) for i in product for j in month], lowBound=0, cat='Integer')

# Objective Function
model += (lpSum(UPC[i]*Pt[i,j] for i in product for j in month) + lpSum(UHC[i]*It[i,j] for i in product for j in month))

# Constraints
for i in product:
    model += It[i,0] == 0

for j in month[1:]:
    model += (Pt['A', j] + It['A', j-1] - It['A', j]) == A_demand[j] # A_Inventory-Balancing Constraints
    model += (Pt['B', j] + It['B', j-1] - It['B', j]) == B_demand[j] # B_Inventory-Balancing Constraints
    model += (Pt['C', j] + It['C', j-1] - It['C', j]) == C_demand[j] # C_Inventory-Balancing Constraints
    model += lpSum(UPH[i] * Pt[i,j] for i in product) <= TPC[j] # Production Hour Capacity Constraints

for i in product:
    model += It[i,4] == 0

# Solve the Problem
model.writeLP("Q1.lp")
model.solve()

# The Status of the Solution
print("Model Status:{}".format(LpStatus[model.status]))

# To Display Optimal Decision Variables & Reduced Cost per Variable
for v in model.variables():
	  print(v.name, "=", v.varValue)

# To Display Optimal Value of Objective Function
print("Objective=", value(model.objective))

Model Status:Optimal
Inventory_('A',_0) = 0.0
Inventory_('A',_1) = 0.0
Inventory_('A',_2) = 0.0
Inventory_('A',_3) = 0.0
Inventory_('A',_4) = 0.0
Inventory_('B',_0) = 0.0
Inventory_('B',_1) = 0.0
Inventory_('B',_2) = 0.0
Inventory_('B',_3) = 0.0
Inventory_('B',_4) = 0.0
Inventory_('C',_0) = 0.0
Inventory_('C',_1) = 62.0
Inventory_('C',_2) = 133.0
Inventory_('C',_3) = 119.0
Inventory_('C',_4) = 0.0
Quantity_('A',_0) = 0.0
Quantity_('A',_1) = 100.0
Quantity_('A',_2) = 110.0
Quantity_('A',_3) = 120.0
Quantity_('A',_4) = 130.0
Quantity_('B',_0) = 0.0
Quantity_('B',_1) = 80.0
Quantity_('B',_2) = 90.0
Quantity_('B',_3) = 100.0
Quantity_('B',_4) = 110.0
Quantity_('C',_0) = 0.0
Quantity_('C',_1) = 182.0
Quantity_('C',_2) = 201.0
Quantity_('C',_3) = 126.0
Quantity_('C',_4) = 51.0
Objective= 26357.0


In [ ]:
from pulp import LpStatus, value

# The Status of the Solution
print("Model Status:{}".format(LpStatus[model.status]))

# Initialize dictionaries to store inventory and quantity values
inventory_values = {}
quantity_values = {}

# Store Optimal Decision Variables
for v in model.variables():
    if "_0" not in v.name:
        if "Inventory" in v.name:
            inventory_values[v.name] = v.varValue
        elif "Quantity" in v.name:
            quantity_values[v.name] = v.varValue

# Extract keys for quantities and inventories
quantity_keys = sorted(quantity_values.keys())
inventory_keys = sorted(inventory_values.keys())

# Print the results in separate columns for quantities and inventories
print("\nOptimal Assignment Plan")
print(f"{'Quantity':<31} {'Inventory'}")
print(f"{'-'*31} {'-'*30}")

max_len = max(len(quantity_keys), len(inventory_keys))

# Print the table with Quantity and Inventory in separate columns
for i in range(max_len):
    quantity_key = quantity_keys[i] if i < len(quantity_keys) else ""
    inventory_key = inventory_keys[i] if i < len(inventory_keys) else ""

    quantity_value = quantity_values.get(quantity_key, "")
    inventory_value = inventory_values.get(inventory_key, "")

    print(f"{quantity_key:<20} {quantity_value:<10} {inventory_key:<20} {inventory_value}")

# Display Optimal Value of Objective Function
print("\nTotal Cost = £", value(model.objective))

Model Status:Optimal

Optimal Assignment Plan
Quantity                        Inventory
------------------------------- ------------------------------
Quantity_('A',_1)    100.0      Inventory_('A',_1)   0.0
Quantity_('A',_2)    110.0      Inventory_('A',_2)   0.0
Quantity_('A',_3)    120.0      Inventory_('A',_3)   0.0
Quantity_('A',_4)    130.0      Inventory_('A',_4)   0.0
Quantity_('B',_1)    80.0       Inventory_('B',_1)   0.0
Quantity_('B',_2)    90.0       Inventory_('B',_2)   0.0
Quantity_('B',_3)    100.0      Inventory_('B',_3)   0.0
Quantity_('B',_4)    110.0      Inventory_('B',_4)   0.0
Quantity_('C',_1)    182.0      Inventory_('C',_1)   62.0
Quantity_('C',_2)    201.0      Inventory_('C',_2)   133.0
Quantity_('C',_3)    126.0      Inventory_('C',_3)   119.0
Quantity_('C',_4)    51.0       Inventory_('C',_4)   0.0

Total Cost = £ 26357.0


##Question D

In [ ]:
# Define Environment & Direction of Optimization
model = LpProblem("Q1", sense=LpMinimize)

# List of Product
product = ['A', 'B', 'C']

# List (TimePeriods)
month = [0,1,2,3,4]

# Parameters and Data
A_demand = {1:100, 2:110, 3:120, 4:130} # Demand data per time period
B_demand = {1:80, 2:90, 3:100, 4:110} # Demand data per time period
C_demand = {1:120, 2:130, 3:140, 4:150} # Demand data per time period
UPC = {'A':20, 'B':30, 'C':10} # Unit Production Cost per time period
UHC = {'A':1, 'B':2, 'C':0.5} # Unit Holding Cost per time period
UPH = {'A':1.2, 'B':1.3, 'C':1} # Production hour per unit
TPC = {1:500, 2:450, 3:400, 4:350} # Production Capacity per time period

# Desicion Variables
Pt = LpVariable.dicts("Quantity", [(i,j) for i in product for j in month], lowBound=0, cat='Integer')
It = LpVariable.dicts("Inventory", [(i,j) for i in product for j in month], lowBound=0, cat='Integer')

# Objective Function
model += (lpSum(UPC[i]*Pt[i,j] for i in product for j in month) + lpSum(UHC[i]*It[i,j] for i in product for j in month))
# Constraints
for i in product:
    model += It[i,0] == 0

for j in month[1:]:
    model += (Pt['A', j] + It['A', j-1] - It['A', j]) == A_demand[j] # A_Inventory-Balancing Constraints
    model += (Pt['B', j] + It['B', j-1] - It['B', j]) == B_demand[j] # B_Inventory-Balancing Constraints
    model += (Pt['C', j] + It['C', j-1] - It['C', j]) == C_demand[j] # C_Inventory-Balancing Constraints
    model += lpSum(UPH[i] * Pt[i,j] for i in product) <= TPC[j] # Production Hour Capacity Constraints

model += lpSum(UHC[i]*It[i,j] for i in product for j in month) <= 500 # Inventory Holding Cost Barrier

for i in product:
    model += It[i,4] == 0

# Solve the Problem
model.writeLP("Q1.lp")
model.solve()

# The Status of the Solution
print("Model Status:{}".format(LpStatus[model.status]))

# To Display Optimal Decision Variables & Reduced Cost per Variable
for v in model.variables():
	  print(v.name, "=", v.varValue)

# To Display Optimal Value of Objective Function
print("Objective=", value(model.objective))

Model Status:Optimal
Inventory_('A',_0) = 0.0
Inventory_('A',_1) = 0.0
Inventory_('A',_2) = 0.0
Inventory_('A',_3) = 0.0
Inventory_('A',_4) = 0.0
Inventory_('B',_0) = 0.0
Inventory_('B',_1) = 0.0
Inventory_('B',_2) = 0.0
Inventory_('B',_3) = 0.0
Inventory_('B',_4) = 0.0
Inventory_('C',_0) = 0.0
Inventory_('C',_1) = 42.0
Inventory_('C',_2) = 113.0
Inventory_('C',_3) = 99.0
Inventory_('C',_4) = 0.0
Quantity_('A',_0) = 0.0
Quantity_('A',_1) = 100.0
Quantity_('A',_2) = 110.0
Quantity_('A',_3) = 120.0
Quantity_('A',_4) = 130.0
Quantity_('B',_0) = 0.0
Quantity_('B',_1) = 80.0
Quantity_('B',_2) = 90.0
Quantity_('B',_3) = 100.0
Quantity_('B',_4) = 110.0
Quantity_('C',_0) = 0.0
Quantity_('C',_1) = 162.0
Quantity_('C',_2) = 201.0
Quantity_('C',_3) = 126.0
Quantity_('C',_4) = 51.0
Objective= 26127.0


In [ ]:
from pulp import LpStatus, value

# The Status of the Solution
print("Model Status:{}".format(LpStatus[model.status]))

# Initialize dictionaries to store inventory and quantity values
inventory_values = {}
quantity_values = {}

# Store Optimal Decision Variables
for v in model.variables():
    if "_0" not in v.name:
        if "Inventory" in v.name:
            inventory_values[v.name] = v.varValue
        elif "Quantity" in v.name:
            quantity_values[v.name] = v.varValue

# Extract keys for quantities and inventories
quantity_keys = sorted(quantity_values.keys())
inventory_keys = sorted(inventory_values.keys())

# Print the results in separate columns for quantities and inventories
print("\nOptimal Assignment Plan")
print(f"{'Quantity':<31} {'Inventory'}")
print(f"{'-'*31} {'-'*30}")

max_len = max(len(quantity_keys), len(inventory_keys))

# Print the table with Quantity and Inventory in separate columns
for i in range(max_len):
    quantity_key = quantity_keys[i] if i < len(quantity_keys) else ""
    inventory_key = inventory_keys[i] if i < len(inventory_keys) else ""

    quantity_value = quantity_values.get(quantity_key, "")
    inventory_value = inventory_values.get(inventory_key, "")

    print(f"{quantity_key:<20} {quantity_value:<10} {inventory_key:<20} {inventory_value}")

# Display Optimal Value of Objective Function
print("\nTotal Cost = £", value(model.objective))


Model Status:Optimal

Optimal Assignment Plan
Quantity                        Inventory
------------------------------- ------------------------------
Quantity_('A',_1)    100.0      Inventory_('A',_1)   0.0
Quantity_('A',_2)    110.0      Inventory_('A',_2)   0.0
Quantity_('A',_3)    120.0      Inventory_('A',_3)   0.0
Quantity_('A',_4)    130.0      Inventory_('A',_4)   0.0
Quantity_('B',_1)    80.0       Inventory_('B',_1)   0.0
Quantity_('B',_2)    90.0       Inventory_('B',_2)   0.0
Quantity_('B',_3)    100.0      Inventory_('B',_3)   0.0
Quantity_('B',_4)    110.0      Inventory_('B',_4)   0.0
Quantity_('C',_1)    162.0      Inventory_('C',_1)   42.0
Quantity_('C',_2)    201.0      Inventory_('C',_2)   113.0
Quantity_('C',_3)    126.0      Inventory_('C',_3)   99.0
Quantity_('C',_4)    51.0       Inventory_('C',_4)   0.0

Total Cost = £ 26127.0
